In [35]:
import pandas
import bqplot
import ipywidgets

Download data from the [WHO](http://apps.who.int/flumart/Default?ReportNo=12) and save locally

In [7]:
flu = pandas.read_csv('who-flu-us-1997-2018.csv', header=2, index_col=[0,3,4])

In [12]:
flu.head()

WHOREGION  \
Country                  Year Week                                  
United States of America 1997 1     Region of the Americas of WHO   
                              2     Region of the Americas of WHO   
                              3     Region of the Americas of WHO   
                              4     Region of the Americas of WHO   
                              5     Region of the Americas of WHO   

                                        FLUREGION       SDATE       EDATE  \
Country                  Year Week                                          
United States of America 1997 1     North America  1996-12-30  1997-01-05   
                              2     North America  1997-01-06  1997-01-12   
                              3     North America  1997-01-13  1997-01-19   
                              4     North America  1997-01-20  1997-01-26   
                              5     North America  1997-01-27  1997-02-02   

                                    SPEC_RECEIVED_NB  SPEC_PROCESSED_NB  AH1  \
Country                  Year Week                                             
United States of America 1997 1                  NaN             2744.0  0.0   
                              2                  NaN             1921.0  0.0   
                              3                  NaN             1519.0  0.0   
                              4                  NaN             1284.0  0.0   
                              5                  NaN             1347.0  0.0   

                                    AH1N12009    AH3  AH5  ANOTSUBTYPED  \
Country                  Year Week                                        
United States of America 1997 1           NaN  247.0  NaN         544.0   
                              2           NaN  215.0  NaN         224.0   
                              3           NaN  193.0  NaN         101.0   
                              4           NaN  118.0  NaN          60.0   
                              5           NaN   77.0  NaN          59.0   

                                    INF_A  BYAMAGATA  BVICTORIA  \
Country                  Year Week                                
United States of America 1997 1       791        NaN        NaN   
                              2       439        NaN        NaN   
                              3       294        NaN        NaN   
                              4       178        NaN        NaN   
                              5       136        NaN        NaN   

                                    BNOTDETERMINED  INF_B  ALL_INF  ALL_INF2  \
Country                  Year Week                                             
United States of America 1997 1                 40     40      831       NaN   
                              2                 18     18      457       NaN   
                              3                 34     34      328       NaN   
                              4                 44     44      222       NaN   
                              5                 82     82      218       NaN   

                                        TITLE  
Country                  Year Week             
United States of America 1997 1     No Report  
                              2     No Report  
                              3     No Report  
                              4     No Report  
                              5     No Report

Make a simple line chart

In [28]:
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

x_data = range(len(flu))
# TODO: Use the real index values: x_data = flu.index.values
y_data = flu.SPEC_RECEIVED_NB

line_chart = bqplot.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc})

ax_x = bqplot.Axis(scale=x_sc)
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')
axes=[ax_x, ax_y]

bqplot.Figure(marks=[line_chart], axes=[ax_x, ax_y])

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='0.2f')]…

Add brushing to just get the x value

In [91]:
pre_table = ipywidgets.HTML(value='hover over the chart...')

index_sel = bqplot.interacts.IndexSelector(scale=x_sc, marks=[line_chart])

def index_change_callback(change):
    pre_table.value = '<pre>{}</pre>'.format(flu.iloc[int(change.new)][
        ['AH1','AH1N12009','AH3', 'AH5', 'ANOTSUBTYPED', 'BYAMAGATA', 'BVICTORIA', 'BNOTDETERMINED']
    ])
    
index_sel.observe(index_change_callback, names=['selected'])

fig_index_sel = bqplot.Figure(marks=[line_chart], axes=[ax_x, ax_y], interaction=index_sel)
ipywidgets.VBox([fig_index_sel, pre_table])

But that's boring; Instead, let's make a second chart that responds to interactions with the first chart:

Could we present that table as a bar chart instead?

In [95]:
def bar_chart(x):
    s = flu.iloc[int(x)][['AH1','AH1N12009','AH3', 'AH5', 'ANOTSUBTYPED', 'BYAMAGATA', 'BVICTORIA', 'BNOTDETERMINED']]

    x_bar_sc = bqplot.OrdinalScale()
    y_bar_sc = bqplot.LinearScale()
    bar_chart = bqplot.Bars(x=s.index.tolist(), y=s.values.tolist(), scales={'x': x_bar_sc, 'y': y_bar_sc})

    ax_bar_x = bqplot.Axis(scale=x_bar_sc)
    ax_bar_y = bqplot.Axis(scale=y_bar_sc, orientation='vertical')

    return bqplot.Figure(marks=[bar_chart], axes=[ax_bar_x, ax_bar_y])

bc = bar_chart(123.4)
bc

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top'…

In [101]:
bc.marks[0].y

array([ 98.,  nan,   4.,  nan, 176.,  nan,  nan, 435.])

Note that we can redefine the values in a chart:

In [102]:
bc.marks[0].y = [1,2,3,4,5,6,7,8]

In [103]:
bc = bar_chart(0)

index_sel = bqplot.interacts.IndexSelector(scale=x_sc, marks=[line_chart])

def index_change_callback(change):
    bc.marks[0].y = flu.iloc[int(change.new)][
        ['AH1','AH1N12009','AH3', 'AH5', 'ANOTSUBTYPED', 'BYAMAGATA', 'BVICTORIA', 'BNOTDETERMINED']
    ].values.tolist()
    
index_sel.observe(index_change_callback, names=['selected'])

fig_index_sel = bqplot.Figure(marks=[line_chart], axes=[ax_x, ax_y], interaction=index_sel)
ipywidgets.VBox([fig_index_sel, bc])